# GNN IMPLEMENTATION

In [154]:
import pandas as pd
import numpy as np
import torch
import json
import sqlite3
import random
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [155]:
db_path = "vectors.db"
connection = sqlite3.connect(db_path)
table_name = "users"
user_df = pd.read_sql_query(f"SELECT * FROM {table_name}", connection)
connection.close()

In [156]:
user_df.head()

,id,username,bio,post_count,follower_count,following_count,last_login
0,1,afrobeezy,b'\xb7\xf8\x18\xbdE{~<\xd0\t\xbf\xbc\x99\x1f\x...,0.054393,0.3125,0.2,2025-01-16 17:54:54
1,2,doey,b'\x98a\xf3\xbd\xdd\xd4E=\x8e\xfd&\xbb1h4\xbcU...,0.000000,0.0000,0.0,1970-01-01 00:00:00
2,3,doeyy,b'\x98a\xf3\xbd\xdd\xd4E=\x8e\xfd&\xbb1h4\xbcU...,0.000000,0.0000,0.0,1970-01-01 00:00:00
3,4,doeyyy,b'\x98a\xf3\xbd\xdd\xd4E=\x8e\xfd&\xbb1h4\xbcU...,0.000000,0.0000,0.0,1970-01-01 00:00:00
4,5,kinha,b'\xe2\xf6\xcd\xbcj\x8dP\xbdE\xa5V=\xda\xd8\x8...,1.000000,1.0000,1.0,2025-01-14 20:43:50


In [157]:
db_path = "vectors.db"
connection = sqlite3.connect(db_path)
table_name = "posts"
posts_df = pd.read_sql_query(f"SELECT * FROM {table_name}", connection)
connection.close()

In [158]:
posts_df['vector'] = posts_df['vector'].apply(lambda x: np.frombuffer(x, dtype=np.float32))

In [159]:
posts_df.head()

,id,vector,category_id,category_name,title,upvote_count,view_count,exit_count,rating_count,average_rating,username,upvoted,bookmarked,emotions
0,12,"[-0.0050623403, -4.739753e-05, 0.03801688, 0.0...",2,Vible,why fit in,0.421569,0.093700,0.000000,0.045455,0.357143,afrobeezy,0,0,"Anger, Determination, Pain, Empowerment, Isola..."
1,13,"[-0.02623921, -0.016178677, 0.0584833, -0.0476...",2,Vible,transcending singularity,0.196078,0.121163,0.000000,0.681818,0.091837,afrobeezy,0,0,None
2,14,"[0.029540151, 0.005941125, 0.017277755, -0.061...",2,Vible,peak performance,0.696078,0.153473,0.011215,0.090909,0.295918,afrobeezy,0,0,None
3,15,"[-0.008260611, 0.05746867, 0.044171125, -0.047...",2,Vible,our existential situation,0.627451,0.119548,0.000000,0.136364,0.336735,afrobeezy,0,0,None
4,16,"[-0.0042227353, 0.028556699, -0.027887717, 0.0...",2,Vible,act now,0.441176,0.111470,0.003738,0.166667,0.244898,afrobeezy,0,0,None


In [160]:
db_path = "vectors.db"
connection = sqlite3.connect(db_path)
table_name = "liked"
liked_df = pd.read_sql_query(f"SELECT * FROM {table_name}", connection)
connection.close()

In [161]:
liked_df.head()

,id,post_id,user_id,liked_at
0,3,33,9,2023-10-26 09:08:32
1,5,36,1,2023-10-28 09:08:35
2,7,36,16,2023-10-28 09:15:57
3,9,52,9,2023-10-30 15:48:46
4,15,50,19,2023-11-01 11:37:04


In [162]:
db_path = "vectors.db"
connection = sqlite3.connect(db_path)
table_name = "viewed"
viewed_df = pd.read_sql_query(f"SELECT * FROM {table_name}", connection)
connection.close()

In [163]:
viewed_df.head()

,id,post_id,user_id,viewed_at
0,10726,1346,5,2024-11-24 20:26:04
1,10739,1357,5,2024-11-24 20:30:47
2,10851,1365,5,2024-11-29 19:09:20
3,10914,1342,5,2024-12-02 09:23:34
4,10917,1367,5,2024-12-02 09:23:50


In [164]:
db_path = "vectors.db"
connection = sqlite3.connect(db_path)
table_name = "rated"
rated_df = pd.read_sql_query(f"SELECT * FROM {table_name}", connection)
connection.close()

In [165]:
rated_df.head()

,id,post_id,user_id,rating_percent,rated_at
0,1,516,1,64,2024-01-11 12:45:41
1,2,160,1,62,2024-01-11 12:47:15
2,3,19,1,20,2024-01-11 12:47:28
3,4,148,1,100,2024-01-11 12:49:35
4,5,513,1,63,2024-01-11 12:50:58


In [166]:
db_path = "vectors.db"
connection = sqlite3.connect(db_path)
table_name = "inspired"
inspired_df = pd.read_sql_query(f"SELECT * FROM {table_name}", connection)
connection.close()

In [167]:
inspired_df.head()

,id,post_id,user_id,inspired_at
0,1,588,1,2023-12-31 09:06:27
1,2,152,1,2023-12-31 09:07:32
2,3,415,1,2023-12-31 09:10:09
3,4,558,1,2023-12-31 09:11:23
4,5,549,1,2023-12-31 09:11:28


In [168]:
import torch
from torch_geometric.data import Data

print("PyTorch Geometric is installed and working!")

PyTorch Geometric is installed and working!


In [169]:
user_df['id'] = pd.to_numeric(user_df['id'], errors='coerce').fillna(-1).astype(int)  # Convert to numeric and replace invalid with -1
posts_df['id'] = pd.to_numeric(posts_df['id'], errors='coerce').fillna(-1).astype(int) 

In [170]:
print(user_df.dtypes)

id                   int32
username            object
bio                 object
post_count         float64
follower_count     float64
following_count    float64
last_login          object
dtype: object


In [171]:
print(posts_df.dtypes)

id                  int32
vector             object
category_id         int64
category_name      object
title              object
upvote_count      float64
view_count        float64
exit_count        float64
rating_count      float64
average_rating    float64
username           object
upvoted             int64
bookmarked          int64
emotions           object
dtype: object


In [172]:
print(interaction_df.dtypes)

Unnamed: 0          int64
post_id            object
user_id            object
viewed_at           int64
liked_at            int64
inspired_at         int64
rating_percent    float64
rated_at            int64
dtype: object


In [173]:
posts_df = posts_df[posts_df['category_id']==8]

In [174]:
posts_df.head()

,id,vector,category_id,category_name,title,upvote_count,view_count,exit_count,rating_count,average_rating,username,upvoted,bookmarked,emotions
974,1341,"[0.037435547, 0.027852902, 0.08847422, 0.00711...",8,Bloom Scroll,it’s just as breathtaking beneath the surface ...,0.303922,0.232633,0.114019,0.151515,0.285714,kinha,0,0,None
975,1342,"[0.0013911794, -0.01670752, 0.059622392, -0.03...",8,Bloom Scroll,flower blooming flowerpower bloomtime floral...,0.058824,0.198708,0.263551,0.106061,0.214286,kinha,0,0,None
976,1343,"[0.032868303, -0.0038568818, 0.11135628, -0.01...",8,Bloom Scroll,nature is calling and i must go nature moody ...,0.225490,0.218094,0.188785,0.075758,0.163265,kinha,0,0,None
977,1344,"[-0.03343923, -0.0067987316, 0.08799345, -0.02...",8,Bloom Scroll,cactus floreciendoflowerbloomingnatureloversfl...,0.088235,0.197092,0.265421,0.045455,0.051020,kinha,0,0,None
978,1345,"[-0.002873235, 0.013968746, 0.11599427, -0.033...",8,Bloom Scroll,my favorite color is nature ••feature by geor...,0.333333,0.271405,0.377570,0.136364,0.469388,kinha,0,0,None


In [175]:
# Step 1: Ensure consistent data types for user_id and post_id
user_df['id'] = user_df['id'].astype(str)  # Ensure user_id is a string
posts_df['id'] = posts_df['id'].astype(str)  # Ensure post_id is a string
interaction_df['user_id'] = interaction_df['user_id'].astype(str).str.strip()  # Convert to string and strip spaces
interaction_df['post_id'] = interaction_df['post_id'].astype(str).str.strip()  # Convert to string and strip spaces

# Step 2: Create user_map and post_map
user_ids = user_df['id'].unique()
post_ids = posts_df['id'].unique()

user_map = {id_: i for i, id_ in enumerate(user_ids)}
post_map = {id_: i + len(user_map) for i, id_ in enumerate(post_ids)}

# Step 3: Create user and post features
user_features = torch.randn(len(user_map), 16)  # Random user features (16 dimensions)
post_features = torch.tensor(np.vstack(posts_df['vector']), dtype=torch.float)  # Post features (384 dimensions)

# Option 1: Resize user features to match post features size (384 dimensions)
user_features_resized = torch.randn(len(user_map), 384)  # Change to 384 dimensions

# Combine user and post features
node_features = torch.cat([user_features_resized, post_features], dim=0)  # Concatenate along the first dimension

# Step 4: Build edges and edge features as before
edges = []
edge_features = []

# Iterate through interaction_df to build edges
for _, row in interaction_df.iterrows():
    user_id = row['user_id']
    post_id = row['post_id']
    
    if user_id in user_map and post_id in post_map:
        user_idx = user_map[user_id]
        post_idx = post_map[post_id]
        edges.append([user_idx, post_idx])

        edge_features.append([
            row.get('liked_at', 0),
            row.get('viewed_at', 0),
            row.get('inspired_at', 0),
            row.get('rating_percent', 0) / 100  # Normalize rating percent
        ])

# Convert edges and edge features to tensors
edges = torch.tensor(edges, dtype=torch.long).t()
edge_features = torch.tensor(edge_features, dtype=torch.float)

# Step 5: Create PyTorch Geometric data object
from torch_geometric.data import Data

data = Data(
    x=node_features,
    edge_index=edges,
    edge_attr=edge_features
)

print(data)

Data(x=[1760, 384], edge_index=[2, 1423], edge_attr=[1423, 4])


In [176]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class RecommendationGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RecommendationGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        
        # Layer 1
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        
        # Layer 2
        x = self.conv2(x, edge_index)
        return x

In [196]:
# Split the data into training and testing sets
train_users, test_users = train_test_split(list(user_map.values()), test_size=0.2, random_state=42)

# Create labels for training (dummy task for now, can be extended)
labels = torch.randint(0, 2, (len(data.x),))  # Binary labels for demo purposes

# Model, optimizer, and loss
model = RecommendationGNN(input_dim=node_features.size(1), hidden_dim=32, output_dim=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(20):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    out = model(data)

    # Loss calculation (use a subset of the data for simplicity)
    loss = criterion(out[train_users], labels[train_users])
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

Epoch 1, Loss: 3.0930042266845703
Epoch 2, Loss: 2.371610403060913
Epoch 3, Loss: 1.7886614799499512
Epoch 4, Loss: 1.3436866998672485
Epoch 5, Loss: 1.0207164287567139
Epoch 6, Loss: 0.7925959229469299
Epoch 7, Loss: 0.6304958462715149
Epoch 8, Loss: 0.5155332684516907
Epoch 9, Loss: 0.4329182207584381
Epoch 10, Loss: 0.37275516986846924
Epoch 11, Loss: 0.32775625586509705
Epoch 12, Loss: 0.29171162843704224
Epoch 13, Loss: 0.2602563798427582
Epoch 14, Loss: 0.23108282685279846
Epoch 15, Loss: 0.2042991667985916
Epoch 16, Loss: 0.1802329272031784
Epoch 17, Loss: 0.15963687002658844
Epoch 18, Loss: 0.14212989807128906
Epoch 19, Loss: 0.1267189085483551
Epoch 20, Loss: 0.11253979057073593


In [197]:
# Get user and post embeddings
embeddings = model(data)
user_embeddings = embeddings[:len(user_map)]  # First part of embeddings corresponds to users
post_embeddings = embeddings[len(user_map):]  # Remaining part corresponds to posts

# Example: Recommend posts for a specific user
def recommend_posts(user_id, top_k=10):
    user_id = str(user_id)  # Convert user_id to string
    user_idx = user_map[user_id]
    user_embedding = user_embeddings[user_idx].unsqueeze(0)  # Shape: [1, embedding_dim]

    # Compute similarity scores
    scores = F.cosine_similarity(user_embedding, post_embeddings)
    top_posts = scores.topk(k=top_k).indices

    # Map indices back to post IDs
    recommended_post_ids = [post_ids[idx] for idx in top_posts]
    return posts_df[posts_df['id'].isin(recommended_post_ids)][['id', 'title']]

# Recommend for a specific user
recommendations = recommend_posts(user_id=16, top_k=10)
print(recommendations)


        id                                              title
975   1342  flower blooming   flowerpower bloomtime floral...
979   1346  soon itll all be ice so lets enjoy the water w...
980   1347  you are the universe infinite limitless eterna...
986   1355  words of wisdom from ‘nature and the human sou...
991   1361  believe in magic because you are magic ✨ trust...
998   1368  your energy is your most expensive currency 💫 ...
999   1369  comparison is the thief of joy remember your j...
1004  1374  the less i needed the better i felt what makes...
1014  1385                       dream the impossible dream 🌀
1020  1393                          it does nature motivation


In [198]:
import torch
import torch.nn.functional as F

# Function to calculate accuracy for the recommendation system
def calculate_accuracy(user_embeddings, post_embeddings, interaction_df, user_map, post_ids, top_k=10):
    correct = 0
    total = 0
    
    # Iterate through each user in user_map
    for user_id in user_map.keys():
        recommended_posts = recommend_posts(user_id, top_k)  # Get top K recommended posts for the user
        relevant_posts = interaction_df[interaction_df['user_id'] == user_id]['post_id'].values  # Posts the user interacted with
        
        # Count how many of the recommended posts are relevant (i.e., they intersect with the relevant posts)
        correct += len(set(recommended_posts['id']).intersection(set(relevant_posts)))
        total += top_k
    
    accuracy = correct / total  # Accuracy as the fraction of relevant posts found in the top_k recommendations
    return accuracy

# Function to recommend posts based on cosine similarity

In [199]:
def recommend_posts(user_id, top_k=10):
    user_id = str(user_id)  # Convert user_id to string
    user_idx = user_map[user_id]
    user_embedding = user_embeddings[user_idx].unsqueeze(0)  # Shape: [1, embedding_dim]

    # Compute similarity scores between the user embedding and all post embeddings
    scores = F.cosine_similarity(user_embedding, post_embeddings)
    
    # Get top K posts based on similarity scores
    top_posts = scores.topk(k=top_k).indices

    # Map indices back to post IDs
    recommended_post_ids = [post_ids[idx] for idx in top_posts]
    return posts_df[posts_df['id'].isin(recommended_post_ids)][['id', 'title']]

# Calculate accuracy
accuracy = calculate_accuracy(user_embeddings, post_embeddings, interaction_df, user_map, post_ids, top_k=10)
print(f"Recommendation Accuracy: {accuracy}")


Recommendation Accuracy: 0.012483829236739975


In [200]:
def mean_reciprocal_rank(recommended_posts, relevant_posts):
    """
    Calculate Mean Reciprocal Rank (MRR) for the recommendations.

    Parameters:
    recommended_posts (list): List of recommended post ids for a user.
    relevant_posts (list): List of relevant post ids for the user.

    Returns:
    float: MRR score.
    """
    mrr = 0
    for i, post in enumerate(recommended_posts):
        if post in relevant_posts:
            mrr = 1 / (i + 1)
            break
    return mrr


In [201]:
def average_precision_at_k(recommended_posts, relevant_posts, k=5):

    relevant_count = 0
    precision_at_k = 0

    for i, post in enumerate(recommended_posts[:k]):
        if post in relevant_posts:
            relevant_count += 1
            precision_at_k += relevant_count / (i + 1)

    return precision_at_k / min(k, len(relevant_posts)) if len(relevant_posts) > 0 else 0


In [202]:
import numpy as np

def ndcg_at_k(recommended_posts, relevant_posts, k=5):
    # Relevance scores: 1 for relevant, 0 for not relevant
    relevance_scores = [1 if post in relevant_posts else 0 for post in recommended_posts[:k]]
    
    # Calculate DCG
    dcg = np.sum([relevance / np.log2(i + 2) for i, relevance in enumerate(relevance_scores)])
    
    # Calculate IDCG (ideal DCG: assuming all relevant posts are ranked at the top)
    ideal_relevance_scores = [1] * min(k, len(relevant_posts))
    idcg = np.sum([relevance / np.log2(i + 2) for i, relevance in enumerate(ideal_relevance_scores)])
    
    # NDCG is the ratio of DCG to IDCG
    return dcg / idcg if idcg != 0 else 0


In [203]:
def mean_reciprocal_rank(recommended_posts, relevant_posts):

    mrr = 0
    for i, post in enumerate(recommended_posts):
        if post in relevant_posts:
            mrr = 1 / (i + 1)
            break
    return mrr

In [204]:
def get_relevant_posts(user_id):
    # Get the relevant posts for a user (posts they liked, viewed, or rated)
    liked_posts = liked_df[liked_df['user_id'] == user_id]['post_id'].tolist()
    viewed_posts = viewed_df[viewed_df['user_id'] == user_id]['post_id'].tolist()
    rated_posts = rated_df[rated_df['user_id'] == user_id]['post_id'].tolist()

    # Combine all relevant posts (unique)
    relevant_posts = list(set(liked_posts + viewed_posts + rated_posts))
    
    return relevant_posts

# Example usage
user_id = 5
relevant_posts = get_relevant_posts(user_id)
print(f"Relevant Posts for User {user_id}: {relevant_posts}")

Relevant Posts for User 5: [12, 14, 15, 16, 19, 23, 30, 31, 33, 34, 35, 36, 38, 39, 40, 43, 44, 46, 47, 49, 2099, 56, 58, 59, 60, 61, 82, 83, 84, 85, 86, 100, 129, 130, 131, 132, 133, 135, 137, 138, 142, 143, 147, 148, 2197, 150, 151, 152, 2200, 154, 153, 158, 159, 160, 161, 164, 165, 166, 167, 168, 169, 171, 173, 175, 176, 177, 2230, 189, 190, 207, 209, 213, 216, 247, 2366, 358, 361, 362, 363, 364, 365, 370, 371, 375, 376, 377, 378, 379, 383, 386, 414, 419, 422, 434, 436, 2495, 449, 459, 461, 2514, 2544, 499, 512, 513, 516, 517, 527, 540, 548, 549, 551, 558, 560, 576, 583, 2633, 2634, 2635, 2637, 590, 2644, 2653, 2654, 2655, 611, 612, 613, 2662, 617, 619, 620, 621, 622, 623, 624, 625, 626, 627, 2671, 629, 2670, 2674, 2672, 2678, 633, 2688, 641, 647, 2696, 2698, 2699, 652, 2701, 654, 2703, 2706, 2707, 2710, 662, 664, 663, 2714, 2716, 691, 694, 695, 696, 697, 698, 702, 2752, 2755, 2757, 2761, 714, 2762, 2764, 2766, 722, 723, 724, 726, 727, 2779, 2784, 2787, 2792, 2802, 2803, 2806, 767, 

In [205]:
# Example: Get recommended posts for a user
recommended_posts = recommend_posts(user_id=5, top_k=10)  # Assume `recommend_posts` is defined
recommended_post_ids = recommended_posts['id'].tolist()  # Convert the recommended posts to a list of IDs
print(f"Recommended Posts for User {user_id}: {recommended_post_ids}")


Recommended Posts for User 5: ['1346', '1349', '1361', '1374', '1381', '1386', '2761', '2795', '2865', '2867']


In [206]:
# Example: Calculate MRR for the user
mrr = mean_reciprocal_rank(recommended_post_ids, relevant_posts)
print(f"MRR for User {user_id}: {mrr}")

# Example: Calculate NDCG for the user
ndcg = ndcg_at_k(recommended_post_ids, relevant_posts, k=5)
print(f"NDCG@5 for User {user_id}: {ndcg}")

# Example: Calculate MAP for the user
map_score = average_precision_at_k(recommended_post_ids, relevant_posts, k=5)
print(f"MAP@5 for User {user_id}: {map_score}")

MRR for User 5: 0
NDCG@5 for User 5: 0.0
MAP@5 for User 5: 0.0


In [207]:
#recommend the posts they have not seen yet
#on what basis are we recommending is it the last 15 interaction based or what